In [6]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [25]:
train = pd.read_json('Data/train.json')
test = pd.read_json('Data/test.json')

In [4]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [57]:
independent = ['bathrooms', 'bedrooms', 'price']

In [58]:
interest_averages = train[independent+['interest_level']].groupby('interest_level').aggregate(np.mean)
print(interest_averages)

                bathrooms  bedrooms        price
interest_level                                  
high             1.116176  1.546496  2700.293045
low              1.238741  1.514759  4176.599142
medium           1.163906  1.622050  3158.767388


In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
model = RandomForestClassifier()
model.fit(train[independent], train['interest_level'])
preds = model.predict(train[independent])
probs = model.predict_proba(train[independent])

In [61]:
probs[:10]

array([[ 0.        ,  0.875     ,  0.125     ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.01745826,  0.82926871,  0.15327303],
       [ 0.        ,  0.80351953,  0.19648047],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.6808189 ,  0.3191811 ],
       [ 0.06284677,  0.76324901,  0.17390422],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.14507873,  0.52430636,  0.33061491],
       [ 0.24075557,  0.38696865,  0.37227579]])

In [62]:
print(preds[0:10])
print(train['interest_level'].values[0:10])

[u'low' u'low' u'low' u'low' u'low' u'low' u'low' u'low' u'low' u'low']
[u'medium' u'low' u'high' u'low' u'low' u'medium' u'low' u'low' u'medium'
 u'low']


In [63]:
pd.crosstab(preds, train['interest_level'].values)        

col_0,high,low,medium
row_0,,,
high,827,248,251
low,2603,33075,8927
medium,409,961,2051


In [47]:
submission = test[['listing_id']]
probs = model.predict_proba(test[['bathrooms', 'bedrooms', 'price']])
submission = pd.concat([submission.reset_index(drop=True), pd.DataFrame(probs, columns=model.classes_)], axis=1)
submission = submission[['listing_id', 'high', 'medium', 'low']]

print(submission.head())
submission.to_csv('Submissions/submission1.csv', index=False)

   listing_id      high    medium       low
0     7142618  0.033739  0.181282  0.784979
1     7210040  0.121331  0.339213  0.539456
2     7103890  0.000000  0.000000  1.000000
3     7143442  0.075461  0.296851  0.627688
4     6860601  0.000000  0.153908  0.846092
